# El precio justo

Hoy crearemos otra pieza del rompecabezas: un ScanningAgent que busca ofertas prometedoras suscribiéndose a feeds RSS.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Inicialización y constantes

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|███████████████████████████████████████████████████████████████████| 5/5 [01:43<00:00, 20.64s/it]


In [4]:
len(deals)

50

In [6]:
deals[44].describe()

"Título: Shark Matrix Self-Emptying Robot Vacuum for $240 for Total members + free shipping\nDetalles: Today only, My Best Buy Total members can get this while it's at the best price we've ever seen. My Best Buy Total is a paid membership, which costs $180 annually. Buy Now at Best Buy\nCaracterísticas: precision home mapping 90 mins runtime Model: RV2320S\nURL: https://www.dealnews.com/products/Shark/Shark-Matrix-Self-Emptying-Robot-Vacuum/462530.html?iref=rss-c196"

In [7]:
system_prompt = """Identifica y resume las 5 ofertas más detalladas de una lista, seleccionando las ofertas que tengan la descripción más detallada y de alta calidad y el precio más claro.
Responde estrictamente en JSON sin explicación, usando este formato. Debes proporcionar el precio como un número derivado de la descripción. Si el precio de una oferta no está claro, no incluyas esa oferta en tu respuesta.
Lo más importante es que respondas con las 5 ofertas que tengan la descripción del producto más detallada con el precio. No es importante mencionar los términos de la oferta; lo más importante es una descripción detallada del producto.
Ten cuidado con los productos que se describen como "$XXX de descuento" o "reducidos en $XXX"; este no es el precio real del producto. Solo responde con productos cuando estés muy seguro del precio.

{"deals": [
    {
        "product_description": "Un resumen claro del producto en 4 o 5 oraciones. Los detalles del artículo son mucho más importantes que el motivo por el que es una buena oferta. Evita mencionar descuentos y cupones; céntrese en el artículo en sí. Debe haber un párrafo de texto para cada artículo que elija.",
        "price": 99.99,
        "url": "la url donde se halla el artículo"
    },
    ...
]}"""

In [8]:
user_prompt = """Responde con las 5 ofertas más prometedoras de esta lista, seleccionando aquellas que tengan la descripción del producto más detallada y de mayor calidad y un precio claro.
Responde estrictamente en JSON, y solo en JSON. Debes reformular la descripción para que sea un resumen del producto en sí, no los términos de la oferta.
Recuerda responder con un párrafo de texto en el campo product_description para cada uno de los 5 artículos que selecciones.
Ten cuidado con los productos que se describen como "$XXX de descuento" o "reducidos en $XXX": este no es el precio real del producto. Solo responde con productos cuando estés muy seguro del precio.

Ofertas:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [9]:
print(user_prompt[:2000])

Responde con las 5 ofertas más prometedoras de esta lista, seleccionando aquellas que tengan la descripción del producto más detallada y de mayor calidad y un precio claro.
Responde estrictamente en JSON, y solo en JSON. Debes reformular la descripción para que sea un resumen del producto en sí, no los términos de la oferta.
Recuerda responder con un párrafo de texto en el campo product_description para cada uno de los 5 artículos que selecciones.

Ofertas:

Título: Camera Last-Minute Savings at Best Buy: Up to 50% off + free shipping
Detalles: Discounted brands include Sony, GoPro, and Canon. We've pictured the Canon EOS R6 Mark II Mirrorless Camera for $1,999.99 (low by $500). Shop Now at Best Buy
Características: 
URL: https://www.dealnews.com/Camera-Last-Minute-Savings-at-Best-Buy-Up-to-50-off-free-shipping/21706615.html?iref=rss-c142

Título: LG C3 Series OLED77C3PUA 77" 4K HDR OLED UHD Smart TV for $1,500 for Total members + free shipping
Detalles: Today only for My Best Buy Tota

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [12]:
len(result.deals)

5

In [15]:
result.deals[2]

Deal(product_description='El Samsung Galaxy S24 FE es un teléfono inteligente con Android que destaca por su rendimiento y versatilidad. Equipado con un sistema de cámara avanzado y un diseño ergonómico, es perfecto para usuarios que buscan calidad en fotografía y funcionalidad diaria. Su opción de intercambio también permite obtener descuentos significativos a la hora de actualizar.', price=249.99, url='https://www.dealnews.com/Samsung-Galaxy-S24-FE-Android-Phones-Up-to-400-off-w-trade-in-bundle-savings-free-shipping/21706601.html?iref=rss-c142')

In [16]:
from agents.scanner_agent import ScannerAgent

In [17]:
agent = ScannerAgent()
result = agent.scan()

In [18]:
result

DealSelection(deals=[Deal(product_description='La LG C3 Series OLED77C3PUA es un televisor inteligente de 77 pulgadas con resolución 4K y HDR. Cuenta con un sistema operativo WebOS 23, una tasa de refresco de 120Hz y soporte para Alexa. Incluye cuatro puertos HDMI 2.1, lo que lo convierte en una opción ideal para quienes buscan una experiencia de visualización excepcional con múltiples dispositivos.', price=1500.0, url='https://www.dealnews.com/products/LG/LG-C3-Series-OLED77-C3-PUA-77-4-K-HDR-OLED-UHD-Smart-TV/449311.html?iref=rss=c142'), Deal(product_description='Los auriculares Beats Studio Pro son inalámbricos y cuentan con tecnología de cancelación de ruido, ideales para disfrutar de música sin distracciones. Están diseñados para ofrecer una calidad de sonido superior y un ajuste cómodo, listo para usar en cualquier momento. Estos auriculares son perfectos para quienes buscan estilo y alto rendimiento en audio.', price=150.0, url='https://www.dealnews.com/products/Beats/Beats-Stud